In [ ]:
import requests
import time
import random
import os
from PIL import Image
import io
import base64
from pathlib import Path

def get_app_url():
    """Détecte automatiquement l'URL de l'application (local vs Render)."""
    # Vérifier si on est en production avec une variable d'environnement
    render_url = os.getenv('RENDER_APP_URL')
    if render_url:
        return render_url.rstrip('/')
    
    # Tenter de détecter si l'app locale tourne
    try:
        response = requests.get('http://localhost:5000/', timeout=5)
        if response.status_code == 200:
            return "http://localhost:5000"
    except requests.exceptions.ConnectionError:
        pass
    
    # Fallback - demander à l'utilisateur
    print("❓ Impossible de détecter l'URL de l'application.")
    print("1. http://localhost:5000 (local)")
    print("2. https://votre-app.onrender.com (production)")
    choice = input("Entrez l'URL complète ou choisissez 1/2: ")
    
    if choice == "1":
        return "http://localhost:5000"
    elif choice == "2":
        return input("Entrez l'URL Render complète: ").rstrip('/')
    else:
        return choice.rstrip('/')

# Configuration automatique
APP_URL = get_app_url()
print(f"🔗 URL de l'application: {APP_URL}")

# Chemins des images de test (adaptés à la structure du projet)
project_root = Path.cwd()
if (project_root / "images_to_test").exists():
    test_images_dir = project_root / "images_to_test"
elif (project_root.parent / "images_to_test").exists():
    test_images_dir = project_root.parent / "images_to_test"
else:
    # Créer des images de test factices si le dossier n'existe pas
    test_images_dir = None
    print("⚠️ Dossier images_to_test non trouvé - utilisation d'images générées")

# Liste des images de test
if test_images_dir:
    TEST_IMAGES = [
        str(test_images_dir / "desert_96.jpg"),
        str(test_images_dir / "meadow_89.jpg"),
        str(test_images_dir / "mountain_87.jpg")
    ]
    # Vérifier que les fichiers existent
    TEST_IMAGES = [img for img in TEST_IMAGES if Path(img).exists()]
    if not TEST_IMAGES:
        print("❌ Aucune image de test trouvée dans le dossier")
        test_images_dir = None

# Classes pour le feedback
CLASSES = ['desert', 'forest', 'meadow', 'mountain']

def create_test_image(size=(600, 600), color='blue'):
    """Crée une image de test factice si pas d'images disponibles."""
    img = Image.new('RGB', size, color=color)
    img_buffer = io.BytesIO()
    img.save(img_buffer, format='JPEG')
    img_buffer.seek(0)
    return img_buffer

def upload_image(image_source):
    """Upload une image et retourne la réponse."""
    try:
        if isinstance(image_source, str):
            # Fichier image existant
            with open(image_source, 'rb') as f:
                files = {'file': ('test.jpg', f, 'image/jpeg')}
                response = requests.post(f"{APP_URL}/predict", files=files, timeout=30)
        else:
            # Buffer d'image générée
            files = {'file': ('test.jpg', image_source, 'image/jpeg')}
            response = requests.post(f"{APP_URL}/predict", files=files, timeout=30)
        
        return response
    except requests.exceptions.Timeout:
        print("⏰ Timeout lors de l'upload")
        return None
    except requests.exceptions.ConnectionError:
        print("❌ Erreur de connexion")
        return None

def send_feedback(predicted_label, confidence, user_choice=None):
    """Envoie un feedback pour une prédiction."""
    user_label = user_choice or random.choice(CLASSES)
    
    feedback_data = {
        "image_data": "data:image/jpeg;base64,fake_data_for_test",
        "predicted_label": predicted_label,
        "predicted_confidence": confidence,
        "user_label": user_label
    }
    
    try:
        response = requests.post(f"{APP_URL}/feedback", json=feedback_data, timeout=10)
        return response
    except requests.exceptions.RequestException as e:
        print(f"❌ Erreur lors de l'envoi du feedback: {e}")
        return None

def test_endpoints():
    """Test rapide des endpoints de base."""
    print("🧪 Test des endpoints de base...")
    
    endpoints = [
        ("/", "Page d'accueil"),
        ("/test", "Endpoint de test"),
        ("/test-slow", "Endpoint lent"),
    ]
    
    for endpoint, description in endpoints:
        try:
            response = requests.get(f"{APP_URL}{endpoint}", timeout=10)
            status = "✅" if response.status_code == 200 else "❌"
            print(f"{status} {description}: {response.status_code}")
        except Exception as e:
            print(f"❌ {description}: Erreur - {e}")

def run_monitoring_test(num_requests=10):
    """Exécute une série de requêtes pour alimenter le dashboard."""
    print(f"🚀 Lancement de {num_requests} requêtes de test...")
    
    # Test des endpoints de base d'abord
    test_endpoints()
    print("\n" + "="*50)
    
    successful_requests = 0
    
    for i in range(num_requests):
        print(f"\n--- Requête {i+1}/{num_requests} ---")
        
        # Sélectionner ou créer une image
        if test_images_dir and TEST_IMAGES:
            image_source = random.choice(TEST_IMAGES)
            print(f"📤 Upload de l'image: {Path(image_source).name}")
        else:
            image_source = create_test_image(
                size=random.choice([(224, 224), (300, 300), (600, 600)]),
                color=random.choice(['red', 'blue', 'green', 'yellow'])
            )
            print(f"📤 Upload d'une image générée")
        
        # Upload de l'image
        response = upload_image(image_source)
        if not response or response.status_code != 200:
            print(f"❌ Erreur lors de l'upload: {response.status_code if response else 'Pas de réponse'}")
            continue
        
        # Simuler les données de prédiction (dans la vraie vie, parser le HTML)
        predicted_label = random.choice(CLASSES)
        confidence = round(random.uniform(0.70, 0.99), 3)
        
        print(f"✅ Prédiction simulée: {predicted_label} (confiance: {confidence})")
        
        # Envoyer un feedback (50% de chance d'être différent de la prédiction)
        user_label = predicted_label if random.random() > 0.5 else random.choice(CLASSES)
        feedback_response = send_feedback(predicted_label, confidence, user_label)
        
        if feedback_response and feedback_response.status_code == 200:
            print(f"✅ Feedback envoyé: {user_label}")
            successful_requests += 1
        else:
            print(f"❌ Erreur lors de l'envoi du feedback")
        
        # Attendre un peu entre les requêtes pour simuler un usage réel
        time.sleep(random.uniform(0.5, 2.0))
    
    print(f"\n🎉 Test terminé ! {successful_requests}/{num_requests} requêtes réussies")
    print(f"📊 Dashboard disponible sur: {APP_URL}/dashboard")
    print("   Identifiants par défaut: admin / admin")

def stress_test(duration_minutes=5):
    """Test de charge pendant une durée donnée."""
    print(f"⚡ Test de charge pendant {duration_minutes} minutes...")
    
    end_time = time.time() + (duration_minutes * 60)
    request_count = 0
    
    while time.time() < end_time:
        # Requêtes rapides
        try:
            response = requests.get(f"{APP_URL}/test", timeout=5)
            if response.status_code == 200:
                request_count += 1
        except:
            pass
        
        time.sleep(0.1)  # 10 requêtes par seconde
    
    print(f"📊 {request_count} requêtes envoyées en {duration_minutes} minutes")

# =============================================================================
# CONFIGURATION DU TEST - Modifiez ces variables selon vos besoins
# =============================================================================

# Type de test à exécuter
TEST_TYPE = "normal"  # Options: "normal", "extended", "stress", "custom"

# Configuration pour le test personnalisé
CUSTOM_REQUESTS = 15
CUSTOM_STRESS_MINUTES = 3

# =============================================================================
# EXÉCUTION DU TEST
# =============================================================================

def run_selected_test():
    """Exécute le test sélectionné."""
    print("🔧 Test de monitoring de l'application Flask")
    print(f"🌐 URL cible: {APP_URL}")
    print(f"🎯 Type de test: {TEST_TYPE}")
    
    if TEST_TYPE == "normal":
        print("📊 Lancement du test normal (10 requêtes)")
        run_monitoring_test(10)
        
    elif TEST_TYPE == "extended":
        print("📊 Lancement du test étendu (30 requêtes)")
        run_monitoring_test(30)
        
    elif TEST_TYPE == "stress":
        print("⚡ Lancement du test de charge (5 minutes)")
        stress_test(5)
        
    elif TEST_TYPE == "custom":
        print(f"🎨 Lancement du test personnalisé ({CUSTOM_REQUESTS} requêtes)")
        run_monitoring_test(CUSTOM_REQUESTS)
        
    else:
        print("⚠️ Type de test non reconnu, lancement du test par défaut...")
        run_monitoring_test(10)

# Exécution automatique
run_selected_test()

🔗 URL de l'application: http://localhost:5000
🔧 Test de monitoring de l'application Flask
🌐 URL cible: http://localhost:5000

Choisissez un test:
1. Test normal (10 requêtes)
2. Test étendu (30 requêtes)
3. Test de charge (5 minutes)
4. Test personnalisé
